In [6]:
%pylab inline
import pandas as pd
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [9]:
p1 = pd.read_excel('raw_data/p1_growth_rate.xlsx')
p2 = pd.read_excel('raw_data/p1_growth_rate.xlsx')
mapping = pd.read_excel('raw_data/mapping_growth_rate.xlsx')
p1['Time'] = arange(0, len(p1)*10, 10)/60
p2['Time'] = arange(0, len(p2)*10, 10)/60
wells = mapping['well'].values

p1 = p1.drop(columns= ['T° 600'])
p2 = p2.drop(columns= ['T° 600'])

p1_blank_mean = p1[mapping['well'][mapping['sample']=='blank'].values].mean(axis=1)
p2_blank_mean = p2[mapping['well'][mapping['sample']=='blank'].values[mapping['well'][mapping['sample']=='blank'].values!='E8']].mean(axis=1)

p1[wells] = p1[wells].apply(lambda x:x-p1_blank_mean)
p2[wells] = p2[wells].apply(lambda x:x-p2_blank_mean)

p1[p1<0] = 0
p2[p2<0] = 0

p1.head()


,Time,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,H3,H4,H5,H6,H7,H8,H9,H10,H11,H12
0,0.000000,0.000375,3.750000e-04,1.375000e-03,0.003375,0.003375,0.002375,0.003375,1.375000e-03,0.002375,...,3.750000e-04,0.003375,0.0,0.0,0.0,3.750000e-04,3.750000e-04,0.002375,1.375000e-03,3.750000e-04
1,0.166667,0.000125,1.250000e-04,1.250000e-04,0.003125,0.003125,0.002125,0.003125,1.125000e-03,0.003125,...,1.250000e-04,0.003125,0.0,0.0,0.0,1.125000e-03,1.250000e-04,0.002125,1.125000e-03,1.250000e-04
2,0.333333,0.000000,8.750000e-04,8.750000e-04,0.002875,0.002875,0.001875,0.002875,8.750000e-04,0.002875,...,0.000000e+00,0.003875,0.0,0.0,0.0,8.750000e-04,0.000000e+00,0.001875,8.750000e-04,0.000000e+00
3,0.500000,0.000000,1.387779e-17,1.387779e-17,0.003000,0.002000,0.002000,0.003000,1.387779e-17,0.003000,...,1.387779e-17,0.004000,0.0,0.0,0.0,1.387779e-17,1.387779e-17,0.002000,1.387779e-17,1.387779e-17
4,0.666667,0.000000,0.000000e+00,0.000000e+00,0.001750,0.002750,0.001750,0.003750,7.500000e-04,0.002750,...,0.000000e+00,0.004750,0.0,0.0,0.0,0.000000e+00,7.500000e-04,0.001750,7.500000e-04,0.000000e+00


In [10]:
def time_to_th(growth_curve, th):
    lg = growth_curve[growth_curve>th].index
    if len(lg)==0:
        lg = inf
    else:
        lg = min(lg)
    
    return lg

sum_data_a = pd.DataFrame(p1[wells].apply(lambda x:time_to_th(x, 0.08)*(10/60))).reset_index()
sum_data_a.columns = ['well', 'time_to_th']
sum_data_a['k'] = 0
sum_data_a['k'] = p1[wells].apply(lambda x:x.tail(50).mean()).values
sum_data_a['plate'] = 1

p2_sum = pd.DataFrame(p2[wells].apply(lambda x:time_to_th(x, 0.08)*(10/60))).reset_index()
p2_sum.columns = ['well', 'time_to_th']
p2_sum['k'] = 0
p2_sum['k'] = p2[wells].apply(lambda x:x.tail(50).mean()).values
p2_sum['plate'] = 2
sum_data_a = pd.concat([sum_data_a, p2_sum])
sum_data_a['sample'] = sum_data_a.apply(lambda x: mapping['sample'][mapping['well']==x['well']].values[0], axis = 1)
sum_data_a['ev/anc'] = sum_data_a['sample'].apply(lambda x:'Evolved' if len(x.split('_'))==2 else 'Ancestor')
sum_data_a['species'] = sum_data_a['sample'].apply(lambda x:x.split('_')[0])
sum_data_a['effective_growth_rate'] = log2(0.08/10**(-4))/sum_data_a['time_to_th']
sum_data_a.head()

,well,time_to_th,k,plate,sample,ev/anc,species,effective_growth_rate
0,A1,13.333333,0.481485,1,Ea,Ancestor,Ea,0.723289
1,A2,10.166667,0.400425,1,IN63,Ancestor,IN63,0.948576
2,A3,13.333333,0.429085,1,Ea,Ancestor,Ea,0.723289
3,A4,10.500000,0.386485,1,IN63,Ancestor,IN63,0.918462
4,A5,12.500000,0.421705,1,Ea_p1A1,Evolved,Ea,0.771508


In [ ]:
sum_data[sum_data['effective_growth_rate']!=inf].groupby(['sample', 'species', 'ev/anc']).mean().groupby(['species','ev/anc']).mean().reset_index().to_excel('proccesed_data/growth_rates.xlsx')